In [2]:
import numpy as np
from scipy import signal

In [80]:
def apply_stride(output, stride_shape):
    strided_output = output[:, :, 0::stride_shape[0], 0::stride_shape[1]]
    return strided_output

In [157]:
def pad(input_tensor, filter_w, filter_h, stride):
    
    batch_size, c, input_w, input_h = input_tensor.shape
    new_w = ((input_w - 1)* stride - input_w + filter_w)//2
    new_h = ((input_h - 1)* stride - input_h + filter_h)//2
    
    pad_width = [(0,0), (0,0), (stride, new_w), (stride,new_h)]
    out = np.pad(input_tensor, pad_width=pad_width, constant_values=0)
    
    return out
    

In [58]:
def conv2(input_tensor, filters, stride_shape, padding=0):
        batch_size, input_c, input_w, input_h = input_tensor.shape
        num_filters, input_c, filter_w, filter_h = filters.shape

        out_w = int((input_w - filter_w + 2*padding)/stride_shape[0]) + 1
        out_h = int((input_h - filter_h + 2*padding)/stride_shape[1]) + 1
        out_c = num_filters

        input_depth = filters.shape[1]
        channel = filters.shape[0]
        output = np.zeros((input_tensor.shape[0], out_c, out_w, out_h))
        
        for m in range(batch_size):
            for k in range(out_c):
                for d in range(input_depth):
                    inputt = input_tensor[m, d]
                    for h in range(out_h):
                        for w in range(out_w):
                            
                            start_x = w*stride_shape[0]
                            stop_x = start_x + filter_w
                            
                            start_y = h*stride_shape[1]
                            stop_y = start_y + filter_h
                            
                            aslice = inputt[start_x: stop_x, start_y: stop_y]
                            kernel = filters[k, d]
                            output[m, k, w, h] += np.sum(aslice * kernel)
        
        return output
                        
                    
        
#         for i in range(out_w):
#             start_x = 0
#             for j in range(out_h):
#                 for k, filter in enumerate(self.filters):
#                     output[:, k, j, i] = np.sum(
#                         input_tensor[:, :, start_y: start_y + step_y, 
#                                         start_x: start_x + step_x] * filter + self.bias[k],
#                                         axis=(1,2,3))
#                 start_x = start_x + self.stride_shape[0]
            
#             start_y += self.stride_shape[1]
        
#         return output

In [59]:
def conv(input_tensor, filters, stride=1, padding=0):
    
    batch_size, input_c, input_w, input_h = input_tensor.shape
    num_filters, input_c, filter_w, filter_h = filters.shape
    
    out_w = int((input_w - filter_w + 2*padding)/stride) + 1
    out_h = int((input_h - filter_h + 2*padding)/stride) + 1
    out_c = num_filters
    
    input_depth = filters.shape[1]
    output = np.zeros((input_tensor.shape[0], out_c, out_w, out_h))
    
    for m in range(input_tensor.shape[0]):
        for c in range(out_c):
            for d in range(input_depth):
                output[m, c] += signal.correlate2d(input_tensor[m, d], filters[c,d], "valid")
    
    return output
    
    

In [88]:
input_tensor = np.random.rand(2,3,7,7)
num_filters = 1
input_depth = input_tensor.shape[1]
filters = np.random.rand(num_filters, input_depth, 3,3)
output_conv1 = conv(input_tensor, filters)

print(output_conv1.shape)

(2, 1, 5, 5)


In [89]:
output_conv2 = conv2(input_tensor, filters, stride_shape=(1,2))
print(output_conv2.shape)

(2, 1, 5, 3)


In [90]:
output_conv1 = apply_stride(output_conv1, (1,2))
output_conv1.shape

(2, 1, 5, 3)

In [91]:
print(np.allclose(output_conv1, output_conv2))

True


In [92]:
print(output_conv2)

[[[[6.04085723 6.21689715 7.15042105]
   [7.83424773 6.98051941 6.51088802]
   [8.11557512 6.7150652  7.27423322]
   [6.6015336  6.96576589 6.63299555]
   [6.83858892 6.0717653  6.14456336]]]


 [[[8.49850472 8.36314121 7.74336605]
   [7.93929108 7.40618425 6.48372688]
   [5.82864341 5.75931222 7.69114587]
   [6.27575497 5.85073068 6.02808037]
   [7.52739357 5.32648495 5.51478062]]]]


In [84]:
print(output_conv1)

[[[[4.839987   5.38670434 7.26868766]
   [6.1550135  6.69991771 6.58112847]
   [5.06901854 6.53453564 6.68890836]]]


 [[[7.13565038 6.82111506 5.73577517]
   [6.84810003 7.23614396 6.14903191]
   [6.25619731 7.08705339 7.50600382]]]]


In [164]:
input_tensor = np.random.rand(2, 1, 3, 3)
filter_w, filter_h = 3,3
stride = 1

print(input_tensor, "\n")
pad_out = pad(input_tensor, filter_w, filter_h, stride)

print(pad_out.shape)
print(pad_out)
# print('\n'.join([' '.join(['{:4}'.format(int(np.ceil(item))) for item in row]) 
#       for row in pad_out]))

[[[[0.3537397  0.44445355 0.83463952]
   [0.11964033 0.7203983  0.19861132]
   [0.81913367 0.17093866 0.22125756]]]


 [[[0.43980427 0.0294004  0.8439703 ]
   [0.42011838 0.0832708  0.15689997]
   [0.9623606  0.76344108 0.24938151]]]] 

(2, 1, 5, 5)
[[[[0.         0.         0.         0.         0.        ]
   [0.         0.3537397  0.44445355 0.83463952 0.        ]
   [0.         0.11964033 0.7203983  0.19861132 0.        ]
   [0.         0.81913367 0.17093866 0.22125756 0.        ]
   [0.         0.         0.         0.         0.        ]]]


 [[[0.         0.         0.         0.         0.        ]
   [0.         0.43980427 0.0294004  0.8439703  0.        ]
   [0.         0.42011838 0.0832708  0.15689997 0.        ]
   [0.         0.9623606  0.76344108 0.24938151 0.        ]
   [0.         0.         0.         0.         0.        ]]]]


In [5]:
a = np.zeros((5, *[2]))